In [1]:
import numpy as np
import os
import cv2

# Define paths to image folders
train_folder = '/home/asreen-mohammad/Desktop/project/train'
validation_folder = '/home/asreen-mohammad/Desktop/project/validation'
test_folder = '/home/asreen-mohammad/Desktop/project/test'

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            if img is not None:
                img = cv2.resize(img, (28, 28))  # Resize image to 28x28 (adjust as needed)
                images.append(img.flatten())  # Flatten the image and add to list
    return images

# Load training images and labels
train_images = load_images_from_folder(train_folder)
train_outputs = np.zeros(len(train_images))  # Assuming binary classification (0 or 1)

# Load validation images and labels
validation_images = load_images_from_folder(validation_folder)
validation_outputs = np.zeros(len(validation_images))  # Assuming binary classification (0 or 1)

# Load test images and labels
test_images = load_images_from_folder(test_folder)
test_outputs = np.zeros(len(test_images))  # Assuming binary classification (0 or 1)

# Initialize weights (+1 for bias)
weights = np.zeros(train_images[0].shape[0] + 1)

# Training the perceptron
learning_rate = 0.1
epochs = 10

for epoch in range(epochs):
    # Training
    train_results = []
    for inputs, output in zip(train_images, train_outputs):
        inputs_with_bias = np.insert(inputs, 0, 1)  # Add bias term
        prediction = np.dot(inputs_with_bias, weights)
        prediction = 1 if prediction > 0 else 0
        weights += learning_rate * (output - prediction) * inputs_with_bias
        train_results.append(prediction == output)
    print(f"Epoch {epoch + 1}: Training accuracy:", np.mean(train_results) if train_results else 0)

    # Validation
    validation_results = []
    for inputs, output in zip(validation_images, validation_outputs):
        inputs_with_bias = np.insert(inputs, 0, 1)  # Add bias term
        prediction = np.dot(inputs_with_bias, weights)
        prediction = 1 if prediction > 0 else 0
        validation_results.append(prediction == output)
    print(f"Epoch {epoch + 1}: Validation accuracy:", np.mean(validation_results) if validation_results else 0)

# Testing
test_results = []
for inputs, output in zip(test_images, test_outputs):
    inputs_with_bias = np.insert(inputs, 0, 1)  # Add bias term
    prediction = np.dot(inputs_with_bias, weights)
    prediction = 1 if prediction > 0 else 0
    test_results.append(prediction == output)
print("Test accuracy:", np.mean(test_results) if test_results else 0)



Epoch 1: Training accuracy: 1.0
Epoch 1: Validation accuracy: 1.0
Epoch 2: Training accuracy: 1.0
Epoch 2: Validation accuracy: 1.0
Epoch 3: Training accuracy: 1.0
Epoch 3: Validation accuracy: 1.0
Epoch 4: Training accuracy: 1.0
Epoch 4: Validation accuracy: 1.0
Epoch 5: Training accuracy: 1.0
Epoch 5: Validation accuracy: 1.0
Epoch 6: Training accuracy: 1.0
Epoch 6: Validation accuracy: 1.0
Epoch 7: Training accuracy: 1.0
Epoch 7: Validation accuracy: 1.0
Epoch 8: Training accuracy: 1.0
Epoch 8: Validation accuracy: 1.0
Epoch 9: Training accuracy: 1.0
Epoch 9: Validation accuracy: 1.0
Epoch 10: Training accuracy: 1.0
Epoch 10: Validation accuracy: 1.0
Test accuracy: 1.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

training_path = "/home/asreen-mohammad/Desktop/project/train"
validation_path = "/home/asreen-mohammad/Desktop/project/validation"
testing_path = "/home/asreen-mohammad/Desktop/project/test"

root = training_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

train_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=True)

root = validation_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

val_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

root = testing_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

test_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(2):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # Calculate training accuracy
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    train_acc = correct / total

    # Calculate validation accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total

    print(f"Epoch [{epoch+1}/10], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = correct / total
print(f"Testing Accuracy: {test_acc:.4f}") 

Skipping /home/asreen-mohammad/Desktop/project/train/saprate.py: cannot identify image file '/home/asreen-mohammad/Desktop/project/train/saprate.py'
Skipping /home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-images-idx3-ubyte: cannot identify image file '/home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-images-idx3-ubyte'
Skipping /home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-images-idx3-ubyte.gz: cannot identify image file '/home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-images-idx3-ubyte.gz'
Skipping /home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-labels-idx1-ubyte: cannot identify image file '/home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-labels-idx1-ubyte'
Skipping /home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-labels-idx1-ubyte.gz: cannot identify image file '/home/asreen-mohammad/Desktop/project/train/MNIST/raw/t10k-labels-idx1-ubyte.gz'
Skipping /home/asreen-mohammad/Desktop/project/train/MNIST/raw/trai

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define transforms for preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load data from folders
train_data = datasets.MNIST('/home/asreen-mohammad/Desktop/project/train', download=True, train=True, transform=transform)
validation_data = datasets.MNIST('/home/asreen-mohammad/Desktop/project/validation', download=True, train=False, transform=transform)
test_data = datasets.MNIST('/home/asreen-mohammad/Desktop/project/test', download=True, train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

# Multi-layer perceptron (MLP) model
mlp = nn.Sequential(
    nn.Linear(784, 128),  # input layer (28x28) -> hidden layer (128)
    nn.ReLU(),
    nn.Linear(128, 64),  # hidden layer (128) -> hidden layer (64)
    nn.ReLU(),
    nn.Linear(64, 10)  # hidden layer (64) -> output layer (10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()
        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/i}')

# Evaluate on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Training Accuracy: {100 * correct / total} %')

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total} %')

# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation Accuracy: {100 * correct / total} %')

Epoch 1, Loss: 0.4334286018418846
Epoch 2, Loss: 0.18082173574385865
Epoch 3, Loss: 0.13175720704288055
Epoch 4, Loss: 0.10466192980379407
Epoch 5, Loss: 0.08882242955790243
Epoch 6, Loss: 0.0778050919236335
Epoch 7, Loss: 0.06961221717871535
Epoch 8, Loss: 0.06115008197516589
Epoch 9, Loss: 0.053315855079507424
Epoch 10, Loss: 0.04903542823637781
Training Accuracy: 98.56333333333333 %
Test Accuracy: 97.56 %
Validation Accuracy: 97.56 %


In [4]:
import tensorflow as tf
import numpy as np
import os

# Assuming OpenCV is installed for image processing
train_dir = "/home/asreen-mohammad/Desktop/project/train"
val_dir = "/home/asreen-mohammad/Desktop/project/validation"
test_dir = "/home/asreen-mohammad/Desktop/project/test"

# Initialize parameters
learning_rate = 0.01
num_epochs = 10
input_shape = (28, 28, 3)  # Assuming images are resized to 28x28 and have 3 channels

# Regularization parameters
l2_regularization = 0.001  # L2 regularization parameter
dropout_rate = 0.2  # Dropout rate
weight_decay = 0.0001  # Weight decay parameter

# Function to load and preprocess data using TensorFlow
def load_data_tf(folder_path, input_shape):
    X = []
    y = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Assuming images are in JPG or PNG format
                img_path = os.path.join(root, file)
                label = 1 if "positive" in root else 0  # Example: folder structure decides the label
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                X.append(img_array)
                y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Load training data
X_train, y_train = load_data_tf(train_dir, input_shape)
X_train = X_train / 255.0

# Load testing data
X_test, y_test = load_data_tf(test_dir, input_shape)
X_test = X_test / 255.0

# Load validation data
X_val, y_val = load_data_tf(val_dir, input_shape)
X_val = X_val / 255.0

# Define the neural network architecture for L2 regularization
model_l2 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization))
])

# Compile the model for L2 regularization
model_l2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with L2 regularization
history_l2 = model_l2.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for L2 regularization on train data
l2_train_accuracy = model_l2.evaluate(X_train, y_train, verbose=0)[1]
print(f"L2 Regularization Train Accuracy: {l2_train_accuracy * 100:.2f}%")

# Evaluate accuracy for L2 regularization on validation data
l2_val_accuracy = model_l2.evaluate(X_val, y_val, verbose=0)[1]
print(f"L2 Regularization Validation Accuracy: {l2_val_accuracy * 100:.2f}%")

# Evaluate accuracy for L2 regularization on test data
l2_test_accuracy = model_l2.evaluate(X_test, y_test, verbose=0)[1]
print(f"L2 Regularization Test Accuracy: {l2_test_accuracy * 100:.2f}%")

# Define the neural network architecture for Dropout regularization
model_dropout = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model for Dropout regularization
model_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with Dropout regularization
history_dropout = model_dropout.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for Dropout regularization on train data
dropout_train_accuracy = model_dropout.evaluate(X_train, y_train, verbose=0)[1]
print(f"Dropout Train Accuracy: {dropout_train_accuracy * 100:.2f}%")

# Evaluate accuracy for Dropout regularization on validation data
dropout_val_accuracy = model_dropout.evaluate(X_val, y_val, verbose=0)[1]
print(f"Dropout Validation Accuracy: {dropout_val_accuracy * 100:.2f}%")

# Evaluate accuracy for Dropout regularization on test data
dropout_test_accuracy = model_dropout.evaluate(X_test, y_test, verbose=0)[1]
print(f"Dropout Test Accuracy: {dropout_test_accuracy * 100:.2f}%")

# Define the neural network architecture for Weight Decay regularization
model_weight_decay = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
])

# Compile the model for Weight Decay regularization
optimizer_weight_decay = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model_weight_decay.compile(optimizer=optimizer_weight_decay, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with Weight Decay regularization
history_weight_decay = model_weight_decay.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for Weight Decay regularization on train data
weight_decay_train_accuracy = model_weight_decay.evaluate(X_train, y_train, verbose=0)[1]
print(f"Weight Decay Train Accuracy: {weight_decay_train_accuracy * 100:.2f}%")

# Evaluate accuracy for Weight Decay regularization on validation data
weight_decay_val_accuracy = model_weight_decay.evaluate(X_val, y_val, verbose=0)[1]
print(f"Weight Decay Validation Accuracy: {weight_decay_val_accuracy * 100:.2f}%")

# Evaluate accuracy for Weight Decay regularization on test data
weight_decay_test_accuracy = model_weight_decay.evaluate(X_test, y_test, verbose=0)[1]
print(f"Weight Decay Test Accuracy: {weight_decay_test_accuracy * 100:.2f}%") 

2024-08-16 18:40:07.062542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 18:40:07.309807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 18:40:07.372343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 18:40:07.770851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 18:40:09.867127: W tensorflow/compiler/tf2

Epoch 1/10


/home/asreen-mohammad/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8426 - loss: 0.5161 - val_accuracy: 1.0000 - val_loss: 0.2848
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2745 - val_accuracy: 1.0000 - val_loss: 0.2459
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2379 - val_accuracy: 1.0000 - val_loss: 0.2155
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2091 - val_accuracy: 1.0000 - val_loss: 0.1907
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.1853 - val_accuracy: 1.0000 - val_loss: 0.1697
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.1651 - val_accuracy: 1.0000 - val_loss: 0.1518
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.1479 - val_accuracy: 1.0000 - val_loss: 0.1362
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.1328 - val_accuracy: 1.0000 - val_loss: 0.1226
Ep

In [6]:
import os
import numpy as np
from PIL import Image

# Define paths to your data folders
training_dir = '/home/asreen-mohammad/Desktop/project/train/'
validation_dir = '/home/asreen-mohammad/Desktop/project/validation/'
testing_dir = '/home/asreen-mohammad/Desktop/project/test/'

# Hyperparameters
epochs = 5
learning_rate = 0.01

# Load and preprocess training, validation, and testing data
training_data = []
training_labels = []
for filename in os.listdir(training_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(training_dir, filename)
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img = img.resize((28, 28))  # Resize to 28x28 pixels
        img_array = np.array(img)
        training_data.append(img_array.flatten())
        training_labels.append(1)  # Example label, adjust based on your dataset

validation_data = []
validation_labels = []
for filename in os.listdir(validation_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(validation_dir, filename)
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img = img.resize((28, 28))  # Resize to 28x28 pixels
        img_array = np.array(img)
        validation_data.append(img_array.flatten())
        validation_labels.append(1)  # Example label, adjust based on your dataset

testing_data = []
testing_labels = []
for filename in os.listdir(testing_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(testing_dir, filename)
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img = img.resize((28, 28))  # Resize to 28x28 pixels
        img_array = np.array(img)
        testing_data.append(img_array.flatten())
        testing_labels.append(1)  # Example label, adjust based on your dataset

# SGD Training
weights_sgd = np.random.randn(len(training_data[0]))
bias_sgd = 0.1
training_accuracy_sgd = []
for epoch in range(epochs):
    correct = 0
    for i in range(len(training_data)):
        prediction = np.dot(weights_sgd, training_data[i]) + bias_sgd
        prediction = 1 if prediction >= 0 else 0
        error = training_labels[i] - prediction
        weights_sgd += learning_rate * error * training_data[i]
        bias_sgd += learning_rate * error
        correct += 1 if prediction == training_labels[i] else 0
    training_accuracy_sgd.append(correct / len(training_data))
    print(f"Epoch {epoch+1}, SGD Training Accuracy: {training_accuracy_sgd[-1]:.2f}")

# Evaluate on validation data with SGD
predicted_labels = (np.dot(np.expand_dims(weights_sgd, axis=0), np.transpose(validation_data)) + bias_sgd) >= 0
predicted_labels = predicted_labels.astype(int)
validation_accuracy_sgd = np.mean(predicted_labels == validation_labels)

print(f"SGD Validation Accuracy: {validation_accuracy_sgd:.2f}")

# Evaluate on testing data with SGD
predicted_labels = (np.dot(np.expand_dims(weights_sgd, axis=0), np.transpose(testing_data)) + bias_sgd) >= 0
testing_accuracy_sgd = np.mean(np.all(predicted_labels == testing_labels, axis=1))


print(f"SGD Testing Accuracy: {testing_accuracy_sgd:.2f}")

# Mini-Batch Gradient Descent (MBGD) Training
weights_mbgd = np.random.randn(len(training_data[0]))
bias_mbgd = 0.1
training_accuracy_mbgd = []
for epoch in range(epochs):
    correct = 0
    for i in range(0, len(training_data), 32):  # Batch size of 32
        batch_data = training_data[i:i+32]
        batch_labels = training_labels[i:i+32]
        for j in range(len(batch_data)):
            prediction = np.dot(weights_mbgd, batch_data[j]) + bias_mbgd
            prediction = 1 if prediction >= 0 else 0
            error = batch_labels[j] - prediction
            weights_mbgd += learning_rate * error * batch_data[j]
            bias_mbgd += learning_rate * error
            correct += 1 if prediction == batch_labels[j] else 0
    training_accuracy_mbgd.append(correct / len(training_data))
    print(f"Epoch {epoch+1}, MBGD Training Accuracy: {training_accuracy_mbgd[-1]:.2f}")

# Evaluate on validation data with MBGD
predicted_labels = (np.dot(np.expand_dims(weights_mbgd, axis=0), np.transpose(validation_data)) + bias_mbgd) >= 0
validation_accuracy_mbgd = np.mean(np.all(predicted_labels == validation_labels, axis=1))
print(f"MBGD Validation Accuracy: {validation_accuracy_mbgd:.2f}")

# Evaluate on testing data with MBGD

predicted_labels = (np.dot(np.expand_dims(weights_mbgd, axis=0), np.transpose(testing_data)) + bias_mbgd) >= 0
testing_accuracy_mbgd = np.mean(np.all(predicted_labels == testing_labels, axis=1))



print(f"MBGD Testing Accuracy: {testing_accuracy_mbgd:.2f}")

# Gradient Descent (GD) Training
weights_gd = np.random.randn(len(training_data[0]))
bias_gd = 0.1
training_accuracy_gd = []
for epoch in range(epochs):
    correct = 0
    for i in range(len(training_data)):
        prediction = np.dot(weights_gd, training_data[i]) + bias_gd
        prediction = 1 if prediction >= 0 else 0
        error = training_labels[i] - prediction
        weights_gd += learning_rate * error * training_data[i]
        bias_gd += learning_rate * error
        correct += 1 if prediction == training_labels[i] else 0
    training_accuracy_gd.append(correct / len(training_data))
    print(f"Epoch {epoch+1}, GD Training Accuracy: {training_accuracy_gd[-1]:.2f}")

#Evaluate on validation data with GD
predicted_labels = (np.dot(np.expand_dims(weights_gd, axis=0), np.transpose(validation_data)) + bias_gd) >= 0
validation_accuracy_GD = np.mean(np.all(predicted_labels == validation_labels, axis=1))
print(f"GD Validation Accuracy: {validation_accuracy_mbgd:.2f}")

# Evaluate on testing data with GD

predicted_labels = (np.dot(np.expand_dims(weights_gd, axis=0), np.transpose(testing_data)) + bias_gd) >= 0
testing_accuracy_GD = np.mean(np.all(predicted_labels == testing_labels, axis=1))

Epoch 1, SGD Training Accuracy: 1.00
Epoch 2, SGD Training Accuracy: 1.00
Epoch 3, SGD Training Accuracy: 1.00
Epoch 4, SGD Training Accuracy: 1.00
Epoch 5, SGD Training Accuracy: 1.00
SGD Validation Accuracy: 1.00
SGD Testing Accuracy: 1.00
Epoch 1, MBGD Training Accuracy: 1.00
Epoch 2, MBGD Training Accuracy: 1.00
Epoch 3, MBGD Training Accuracy: 1.00
Epoch 4, MBGD Training Accuracy: 1.00
Epoch 5, MBGD Training Accuracy: 1.00
MBGD Validation Accuracy: 1.00
MBGD Testing Accuracy: 1.00
Epoch 1, GD Training Accuracy: 1.00
Epoch 2, GD Training Accuracy: 1.00
Epoch 3, GD Training Accuracy: 1.00
Epoch 4, GD Training Accuracy: 1.00
Epoch 5, GD Training Accuracy: 1.00
GD Validation Accuracy: 1.00
